#  <font color=#e41b52> CoxPH multivariant 
------

In [1]:

import os
import sys
sys.path.insert(0, '/home/labs/hornsteinlab/hadarkl/Hadar/primals/code')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from lifelines import CoxPHFitter, KaplanMeierFitter, statistics
from lifelines.utils import k_fold_cross_validation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

#---------------------------------------------------------------------------
#loaders
%load_ext autoreload
%autoreload 2

from configs.constants import seed
from configs.proteins_lists import SIGNIFICANT_PROTEIN_COX_UNI_LOW_CORR_DATA
from utils.utils_protein_processing import scale_data
from utils.survival_analysis_utils import dichotomize_proteins_by_median

-----------------------------------------------------------------------------------------------------------------

## 📎 <font color=#df208f>Load Data  


### <font color = "pink" >Data:
- log before combat 

- no missing values

In [74]:
# # log befor combat applay - no missing values
# df_combat_dis = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/rep_dis/log_before_combat_discovery.csv')
# #df_combat_dis = df_combat_dis.set_index('sample number')
# df_combat_rep = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/rep_dis/log_before_combat_replication.csv')
# #df_combat_rep = df_combat_rep.set_index('sample number')



### <font color = "pink" >Data - In my current analysis I Decided to continue with this data 
- log before combat 
- low corr data(remove r < 0.7)
- no missing values
- clinical data (Survival_from_onset (months), Status dead=1, Age Onset ,ALSFRS score,ALSFRS_slope)

In [102]:

discovery= pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_before_combat_discovery_low_collinearity.csv')
discovery= discovery.rename(columns={'sample_number': 'sample number'})
discovery = discovery.set_index('sample number')
replication = pd.read_csv('/home/labs/hornsteinlab/hadarkl/Hadar/primals/code/data/raw_data/proteins/DIANN/COMBAT/log_before_combat_replication_low_collinearity.csv')
replication= replication.rename(columns={'sample_number': 'sample number'})
replication = replication.set_index('sample number')


### Dicotomized data  :

In [2]:
def dichotomize_by_discovery_medians(df_discovery, df_replication, exclude_columns=None):
    """
    Dichotomize both discovery and replication datasets based on medians from the discovery cohort.

    :param df_discovery: DataFrame for discovery cohort (used to calculate medians).
    :param df_replication: DataFrame for replication cohort (to apply same medians).
    :param exclude_columns: Columns to exclude from dichotomization (e.g., clinical columns).
    :return: Tuple of (dichotomized discovery DataFrame, dichotomized replication DataFrame, median dictionary).
    """
    if exclude_columns is None:
        exclude_columns = []

    medians = {}
    df_dis_bin = df_discovery.copy()
    df_rep_bin = df_replication.copy()

    for col in df_discovery.columns:
        if col not in exclude_columns and pd.api.types.is_numeric_dtype(df_discovery[col]):
            median_val = df_discovery[col].median()
            medians[col] = median_val

            df_dis_bin[f"{col}_dichotomized"] = (df_discovery[col] > median_val).astype(int)
            df_rep_bin[f"{col}_dichotomized"] = (df_replication[col] > median_val).astype(int)

    return df_dis_bin, df_rep_bin, medians


In [ ]:
dichotomize_by_discovery_medians(discovery, replication)

z-scoring

In [94]:
discovery = scale_data(discovery, scaling="zscore").reset_index('sample number')

replication = scale_data(replication, scaling="zscore").reset_index('sample number')

Z-score normalization applied.
Z-score normalization applied.


### <font color = "pink" > clinical data

In [103]:
#clinical data
clinical_discovery = pd.read_csv('/home/labs/hornsteinlab/Collaboration/PRIMALS/Serum/Hadar/primals/code/data/raw_data/clinical_data/clinical_data_discovery.csv')
clinical_discovery = clinical_discovery[['sample number','Survival_from_onset (months)','Status dead=1','ALSFRS_slope (-unit/month)', 'ALSFRS score (unit)']]
#clinical_dis =clinical_dis.set_index('sample number')
clinical_replication = pd.read_csv('/home/labs/hornsteinlab/Collaboration/PRIMALS/Serum/Hadar/primals/code/data/raw_data/clinical_data/clinical_data_replication.csv')
clinical_replication= clinical_replication[['sample number','Survival_from_onset (months)','Status dead=1','ALSFRS_slope (-unit/month)', 'ALSFRS score (unit)']]
#clinical_rep = clinical_rep.set_index('sample number')

merge protein data with clinical data:

In [104]:
df_discovery_merge = pd.merge(discovery, clinical_discovery, on='sample number', how='left')
df_discovery_merge = df_discovery_merge.set_index('sample number')
df_replication_merge = pd.merge(replication, clinical_replication, on='sample number', how='left')
df_replication_merge = df_replication_merge.set_index('sample number')

-------------------------------------------------------------------------------------------------


### <font color ='pink'> COX multivariant - with cross validation

In [105]:
df_discovery_merge

,A1BG,ABCA1,ABCE1,ABHD14B,ABI3BP,ACACA,ACAN,ACAT2,ACE,ACO1,...,ZNF550,ZNF639,ZNF705B.ZNF705D,ZNF728,ZNF839,ZYX,Survival_from_onset (months),Status dead=1,ALSFRS_slope (-unit/month),ALSFRS score (unit)
sample number,,,,,,,,,,,,,,,,,,,,,
711,32.459080,17.015121,23.159929,14.627908,23.570337,25.015745,22.101562,17.787973,22.870462,20.546500,...,23.598555,24.990916,23.267159,17.848860,17.129158,19.255654,21,1,1.250000,43
743,32.305205,17.015121,23.127965,15.942541,23.631257,25.080328,21.353149,18.724461,23.720225,20.470663,...,25.065112,24.045357,22.790018,17.436486,18.800393,20.927512,44,1,0.666667,36
744,32.323772,21.673171,23.423031,16.362781,23.206705,25.861563,21.295759,19.580121,23.053794,21.162831,...,24.255886,24.192975,23.732027,17.541730,18.486880,21.226167,62,1,0.074074,46
747,32.142225,17.015121,23.306386,15.440992,23.427468,25.292285,20.694528,17.851310,23.438699,20.050952,...,23.266365,25.216788,22.064022,17.749950,18.277130,19.581688,33,1,0.250000,45
749,32.424568,17.015121,23.580672,13.931568,23.564803,25.217729,21.421111,18.683962,23.054345,21.299404,...,24.660324,24.320203,21.744287,18.331105,16.938676,18.920264,49,1,0.500000,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,32.136014,17.015121,22.412046,12.720938,23.556374,25.468611,20.581506,13.182273,23.020305,20.407781,...,25.893952,24.678790,20.484566,20.205246,14.810356,19.434697,73,1,0.222222,40
1932,32.048399,17.015121,22.806363,16.511968,22.908193,24.237360,20.964760,13.182273,22.598951,19.070995,...,24.650367,25.432324,23.853604,20.570709,17.674666,23.214120,44,0,1.000000,36
2053,31.308049,17.015121,22.266985,15.504587,23.160253,22.925214,20.461051,18.019536,23.034526,20.098971,...,26.410453,23.718491,20.484566,19.659673,14.810356,21.730581,61,1,0.333333,40


### Discovery: 

In [106]:

data = df_discovery_merge[SIGNIFICANT_PROTEIN_COX_UNI_LOW_CORR_DATA + ['Survival_from_onset (months)', 'Status dead=1']]

# Define covariates
covariate_columns = [col for col in data.columns if col not in ['Status dead=1', 'Survival_from_onset (months)']]
print(f"Initial covariates: {len(covariate_columns)}")

# Drop rows with missing data
data = data.dropna(subset=covariate_columns + ['Status dead=1', 'Survival_from_onset (months)'])

# Filter out low-variance features
selector = VarianceThreshold(threshold=0.0)
selector.fit(data[covariate_columns])
covariate_columns = [covariate_columns[i] for i in range(len(covariate_columns)) if selector.get_support()[i]]
print(f"Covariates after variance filtering: {len(covariate_columns)}")

# Initialize CoxPHFitter
cph = CoxPHFitter(alpha=0.05, penalizer=0.0,l1_ratio=0.8)

# Cross-validation
scores = k_fold_cross_validation(
    cph, data[[*covariate_columns, 'Status dead=1', 'Survival_from_onset (months)']],
    duration_col='Survival_from_onset (months)',
    event_col='Status dead=1',
    k=3, scoring_method="concordance_index"
)

# Output fold scores
for i, score in enumerate(scores, 1):
    print(f"Fold {i} Concordance Index: {score:.4f}")

avg_score = np.mean(scores)
print(f"\nAverage Concordance Index: {avg_score:.4f}")

# Fit final model
cph.fit(data[covariate_columns + ['Status dead=1', 'Survival_from_onset (months)']],
        duration_col='Survival_from_onset (months)', event_col='Status dead=1')

# Print summary
cph.print_summary()




Initial covariates: 32
Covariates after variance filtering: 32
Fold 1 Concordance Index: 0.7788
Fold 2 Concordance Index: 0.7541
Fold 3 Concordance Index: 0.7784

Average Concordance Index: 0.7704


<lifelines.CoxPHFitter: fitted with 181 total observations, 28 right-censored observations>
             duration col = 'Survival_from_onset (months)'
                event col = 'Status dead=1'
      baseline estimation = breslow
   number of observations = 181
number of events observed = 153
   partial log-likelihood = -570.44
         time fit was run = 2025-04-22 18:20:21 UTC

---
                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                          
CILP2                       -0.52       0.59       0.20            -0.92            -0.13                 0.40                 0.88
REG1A                        0.21       1.23       0.21            -0.20             0.62                 0.82                 1.87
GRN                          0.63       1.88       0.45            -0.25             1.51                 0.78                 4.51
MRC1                         0.40       1.49       0.46            -0.49             1.29                 0.61                 3.63
IGFBP2                      -0.02       0.98       0.28            -0.56             0.52                 0.57                 1.69
ADAM15                       0.24       1.28       0.15            -0.05             0.54                 0.95                 1.72
EPHB4                        0.25       1.28       0.11             0.03             0.47                 1.03                 1.60
IGFALS                       0.99       2.68       0.43             0.15             1.82                 1.16                 6.18
VSIG4                        0.28       1.32       0.21            -0.14             0.70                 0.87                 2.00
FGL1                         0.18       1.19       0.08             0.02             0.33                 1.02                 1.39
SPON1                       -0.38       0.69       0.35            -1.07             0.32                 0.34                 1.37
APMAP                        0.60       1.82       0.25             0.10             1.09                 1.11                 2.98
PON1                        -1.04       0.35       0.29            -1.61            -0.47                 0.20                 0.62
PTPRJ                       -0.61       0.55       0.30            -1.19            -0.02                 0.30                 0.98
IGDCC4                      -0.06       0.94       0.15            -0.36             0.24                 0.70                 1.28
EXTL2                        0.62       1.86       0.42            -0.20             1.45                 0.82                 4.25
SHBG                         0.64       1.90       0.25             0.15             1.14                 1.16                 3.12
IGF1                        -0.38       0.69       0.27            -0.90             0.15                 0.41                 1.16
APOF                         0.70       2.02       0.34             0.03             1.37                 1.04                 3.94
CTSB                         0.08       1.08       0.27            -0.45             0.60                 0.64                 1.83
LRG1                        -0.58       0.56       0.31            -1.18             0.02                 0.31                 1.02
FKBP1A                       0.31       1.36       0.11             0.09             0.52                 1.10                 1.69
CNTN3                       -0.47       0.62       0.28            -1.02             0.07                 0.36                 1.08
GHR                         -0.06       0.94       0.06            -0.19             0.06                 0.83                 1.06
SECTM1                      -0.17       0.84       0.10            -0.38             0.03                 0.68                 1.03
CDH1                         1.13       3.10    

Fdr correction:

In [70]:

# Get the summary DataFrame
summary_df = cph.summary.copy()

# Sort by p-values (ascending)
summary_df = summary_df.sort_values(by="p")

# Apply FDR correction
summary_df["FDR_p"] = multipletests(summary_df["p"], method='fdr_bh')[1]

# Filter for significant results after FDR
significant_fdr = summary_df[summary_df["FDR_p"] < 0.05]

# View
print(f"{len(significant_fdr)} features significant after FDR correction (q < 0.05)")
display(significant_fdr[["exp(coef)", "p", "FDR_p"]])


10 features significant after FDR correction (q < 0.05)


,exp(coef),p,FDR_p
covariate,,,
ALSFRS_slope (-unit/month),3.695982,7.411802e-18,2.445895e-16
ALSFRS score (unit),1.075471,8.763671e-06,1.446006e-04
PON1,0.071054,2.022086e-04,2.224295e-03
CDH1,10.159819,1.710646e-03,1.411283e-02
NPC2,0.095006,3.002835e-03,1.981871e-02
SHBG,9.958566,6.449006e-03,3.546953e-02
CILP2,0.059608,8.655198e-03,3.741875e-02
FKBP1A,4.549931,9.315569e-03,3.741875e-02
APMAP,14.167652,1.020511e-02,3.741875e-02


### Replication

In [71]:
data = df_replication_merge[SIGNIFICANT_PROTEIN_COX_UNI_LOW_CORR_DATA + ['Survival_from_onset (months)', 'Status dead=1']]

data

,CILP2,REG1A,GRN,MRC1,IGFBP2,ADAM15,EPHB4,IGFALS,VSIG4,FGL1,...,CDH1,NPC2,FETUB,ICOSLG,FRMPD3,ALSFRS_slope (-unit/month),Age Onset (years),ALSFRS score (unit),Survival_from_onset (months),Status dead=1
sample number,,,,,,,,,,,,,,,,,,,,,
705,0.700028,0.375816,0.785663,0.245037,0.346687,0.431203,0.989076,0.506732,0.546200,0.484788,...,0.489650,0.456014,0.399191,0.574933,0.652117,0.583333,73,41,40,1
712,0.592460,0.674960,0.771732,0.676869,0.347658,0.558516,0.296910,0.610425,0.627248,0.730658,...,0.332228,0.382287,0.175297,0.481046,0.478521,0.416667,63,43,55,1
717,0.646534,0.496709,0.653193,0.536730,0.619044,0.631754,0.799611,0.667761,0.624905,0.411067,...,0.297367,0.503947,0.363807,0.589000,0.530275,3.666667,70,37,5,1
735,0.630524,0.628366,0.303943,0.676900,0.322019,0.736801,0.177538,0.739209,0.449121,0.240776,...,0.494465,0.290268,0.378014,0.726561,0.536512,3.500000,65,34,9,1
746,0.728460,0.610682,0.546216,0.334345,0.120903,0.370231,0.212346,0.574715,0.429505,0.217003,...,0.494330,0.523857,0.775605,0.404918,0.611612,1.000000,37,44,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,0.801045,0.464779,0.105633,0.400919,0.517523,0.123048,0.297398,0.204091,0.138763,0.212033,...,0.486457,0.429489,0.299530,0.505212,0.564939,0.214286,64,39,80,0
1900,0.437165,0.529784,0.385404,0.309956,0.587815,0.260327,0.259051,0.541002,0.643409,0.624031,...,0.637823,0.413668,0.758853,0.685412,0.795786,0.500000,78,45,26,1
1912,0.860095,0.113002,0.195886,0.363431,0.341043,0.425429,0.283914,0.454725,0.534935,0.215883,...,0.371862,0.301541,0.967916,0.886805,0.597614,1.600000,55,40,48,0


In [ ]:



# Define covariates
covariate_columns = [col for col in data.columns if col not in ['Status dead=1', 'Survival_from_onset (months)']]
print(f"Initial covariates: {len(covariate_columns)}")

# Drop rows with missing data
data = data.dropna(subset=covariate_columns + ['Status dead=1', 'Survival_from_onset (months)'])

# Filter out low-variance features
selector = VarianceThreshold(threshold=0.0)
selector.fit(data[covariate_columns])
covariate_columns = [covariate_columns[i] for i in range(len(covariate_columns)) if selector.get_support()[i]]
print(f"Covariates after variance filtering: {len(covariate_columns)}")

# Initialize CoxPHFitter
cph = CoxPHFitter(alpha=0.05, penalizer=0.0)

# Cross-validation
scores = k_fold_cross_validation(
    cph, data[[*covariate_columns, 'Status dead=1', 'Survival_from_onset (months)']],
    duration_col='Survival_from_onset (months)',
    event_col='Status dead=1',
    k=3, scoring_method="concordance_index"
)

# Output fold scores
for i, score in enumerate(scores, 1):
    print(f"Fold {i} Concordance Index: {score:.4f}")

avg_score = np.mean(scores)
print(f"\nAverage Concordance Index: {avg_score:.4f}")

# Fit final model
cph.fit(data[covariate_columns + ['Status dead=1', 'Survival_from_onset (months)']],
        duration_col='Survival_from_onset (months)', event_col='Status dead=1')

# Print summary
cph.print_summary()

# Check proportional hazards assumption
cph.check_assumptions(data, p_value_threshold=0.05)

# Filter for significant proteins (p < 0.05)
summary_df = cph.summary
significant = summary_df[summary_df['p'] < 0.05]

# Plot forest plot for significant proteins only
if not significant.empty:
    plt.figure(figsize=(6, len(significant) * 0.6))
    hr = significant['exp(coef)']
    ci_lower = significant['exp(coef) lower 95%']
    ci_upper = significant['exp(coef) upper 95%']

    plt.errorbar(x=hr, y=significant.index, xerr=[hr - ci_lower, ci_upper - hr],
                 fmt='o', color='blue', ecolor='gray', capsize=4)

    plt.axvline(x=1, color='red', linestyle='--', label='HR = 1 (no effect)')
    plt.xlabel('Hazard Ratio (HR)')
    plt.title('Significant Features (p < 0.05)')
    plt.grid(True, axis='x')
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No significant features (p < 0.05) found.")


## <font color ='pink'> Cox without croos valitation 

In [35]:
data = df_discovery_merge[SIGNIFICANT_PROTEIN_COX_UNI_LOW_CORR_DATA + ['Survival_from_onset (months)', 'Status dead=1']]


In [36]:


# Define covariate columns (exclude survival and event status columns)
covariate_columns = [col for col in data.columns if col not in ['Status dead=1', 'Survival_from_onset (months)']]

# Initialize the CoxPHFitter

cph = CoxPHFitter(alpha=0.05, penalizer=0.0, strata=None)

# Fit the Cox model on the entire dataset
cph.fit(data[covariate_columns + ['Status dead=1', 'Survival_from_onset (months)']], 
        duration_col='Survival_from_onset (months)', 
        event_col='Status dead=1')

# Output the model summary, including p-values and coefficients
cph.print_summary()

# Extract relevant results
cox_results = cph.summary[['coef', 'exp(coef)', 'p', 'exp(coef) lower 95%', 'exp(coef) upper 95%']]
cox_results.rename(columns={
    'coef': 'Coefficient',
    'exp(coef)': 'Hazard Ratio',
    'p': 'P-Value',
    'exp(coef) lower 95%': 'CI Lower 95%',
    'exp(coef) upper 95%': 'CI Upper 95%'
}, inplace=True)



<lifelines.CoxPHFitter: fitted with 181 total observations, 28 right-censored observations>
             duration col = 'Survival_from_onset (months)'
                event col = 'Status dead=1'
      baseline estimation = breslow
   number of observations = 181
number of events observed = 153
   partial log-likelihood = -569.91
         time fit was run = 2025-04-22 14:04:22 UTC

---
                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                          
CILP2                       -0.37       0.69       0.14            -0.64            -0.09                 0.53                 0.91
REG1A                        0.12       1.12       0.12            -0.11             0.35                 0.89                 1.42
GRN                          0.15       1.16       0.13            -0.11             0.41                 0.90                 1.51
MRC1                         0.13       1.14       0.13            -0.13             0.39                 0.88                 1.48
IGFBP2                      -0.03       0.97       0.19            -0.41             0.35                 0.66                 1.42
ADAM15                       0.26       1.30       0.15            -0.02             0.55                 0.98                 1.73
EPHB4                        0.27       1.30       0.12             0.03             0.50                 1.03                 1.65
IGFALS                       0.41       1.51       0.17             0.08             0.74                 1.09                 2.10
VSIG4                        0.16       1.17       0.13            -0.09             0.41                 0.91                 1.51
FGL1                         0.29       1.33       0.12             0.05             0.52                 1.05                 1.69
SPON1                       -0.14       0.87       0.14            -0.42             0.13                 0.66                 1.14
APMAP                        0.31       1.37       0.12             0.07             0.56                 1.08                 1.74
PON1                        -0.46       0.63       0.12            -0.70            -0.22                 0.50                 0.80
PTPRJ                       -0.27       0.77       0.12            -0.50            -0.03                 0.60                 0.97
IGDCC4                      -0.07       0.93       0.14            -0.34             0.20                 0.71                 1.22
EXTL2                        0.19       1.21       0.15            -0.10             0.48                 0.90                 1.62
SHBG                         0.43       1.53       0.16             0.12             0.73                 1.13                 2.08
IGF1                        -0.26       0.77       0.17            -0.60             0.07                 0.55                 1.07
APOF                         0.29       1.33       0.13             0.03             0.55                 1.03                 1.73
CTSB                         0.03       1.03       0.13            -0.22             0.28                 0.80                 1.32
LRG1                        -0.27       0.77       0.14            -0.54             0.00                 0.58                 1.00
FKBP1A                       0.25       1.28       0.10             0.06             0.44                 1.06                 1.55
CNTN3                       -0.23       0.79       0.14            -0.51             0.05                 0.60                 1.05
GHR                         -0.12       0.89       0.12            -0.36             0.12                 0.70                 1.12
SECTM1                      -0.23       0.79       0.14            -0.51             0.04                 0.60                 1.04
CDH1                         0.42       1.52    

Fdr correction:

In [37]:
# Sort and apply FDR correction
cox_results_sorted = cox_results.sort_values(by="P-Value").copy()
cox_results_sorted["FDR q-value"] = multipletests(cox_results_sorted["P-Value"], method='fdr_bh')[1]

# Filter significant
cox_results_fdr = cox_results_sorted[cox_results_sorted["FDR q-value"] < 0.05]

# Display
print(f"{len(cox_results_fdr)} features are significant after FDR correction (q < 0.05).")
display(cox_results_fdr)

10 features are significant after FDR correction (q < 0.05).


,Coefficient,Hazard Ratio,P-Value,CI Lower 95%,CI Upper 95%,FDR q-value
covariate,,,,,,
ALSFRS_slope (-unit/month),1.307246,3.695982,7.411802e-18,2.744531,4.977274,2.445895e-16
ALSFRS score (unit),0.072759,1.075471,8.763671e-06,1.041520,1.110529,1.446006e-04
PON1,-0.460335,0.631072,2.022086e-04,0.495040,0.804486,2.224295e-03
CDH1,0.418035,1.518974,1.710646e-03,1.169758,1.972442,1.411283e-02
NPC2,-0.414962,0.660366,3.002835e-03,0.502058,0.868590,1.981871e-02
SHBG,0.425889,1.530951,6.449006e-03,1.126886,2.079899,3.546953e-02
CILP2,-0.366959,0.692838,8.655198e-03,0.526813,0.911186,3.741875e-02
FKBP1A,0.250223,1.284312,9.315569e-03,1.063553,1.550893,3.741875e-02
APMAP,0.314855,1.370060,1.020511e-02,1.077479,1.742089,3.741875e-02
